## Preambule

In [15]:
import numpy as np
import pandas as pd
import xarray as xr
import webcolors

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Input

In [16]:
rules_b = ['GF', 'PC', 'PCC', 'AP', 'ECPC', 'GDR']
rulecolors = ['sienna', 'goldenrod', 'forestgreen', 'mediumvioletred', 'tomato', 'steelblue']
rulenames = ['Grandfathering', 'Per capita', 'Per capita convergence', 'Ability to pay', "Equal cumulative per capita", 'Greenhouse Development Rights']

## Read data

In [17]:
xr_dataread = xr.open_dataset("K:/ECEMF/T5.2/xr_dataread.nc")
#xr_2030 = xr.open_dataset("X:/user/dekkerm/Projects/ECEMF_T5.2/Data/xr_alloc_2030.nc")

In [18]:
df_eng = pd.read_csv('X:/user/dekkerm/Data/ENGAGE/PolicyScenarios/engage-internal_snapshot_1695642798.csv')
df_eng = df_eng[df_eng.Variable == 'Emissions|CO2']
df_eng = df_eng.reset_index(drop=True)
regions_df = np.array(df_eng.Region)
regions_df[regions_df == "Argentine Republic"] = 'ARG'
regions_df[regions_df == "Canada"] = 'CAN'
regions_df[regions_df == "Commonwealth of Australia"] = 'AUS'
regions_df[regions_df == "Federative Republic of Brazil"] = 'BRA'
regions_df[regions_df == "People's Repulic of China"] = 'CHN'
regions_df[regions_df == "European Union (28 member countries)"] = 'EU'
regions_df[regions_df == "Republic of India"] = 'IND'
regions_df[regions_df == "Republic of Indonesia"] = 'IDN'
regions_df[regions_df == "State of Japan"] = 'JPN'
regions_df[regions_df == "Russian Federation"] = 'RUS'
regions_df[regions_df == "Kingdom of Saudi Arabia"] = 'SAU'
regions_df[regions_df == "Republic of South Africa"] = 'ZAF'
regions_df[regions_df == "Republic of Korea (South Korea)"] = 'KOR'
regions_df[regions_df == "United Mexican States"] = 'MEX'
regions_df[regions_df == "Republic of Turkey"] = 'TUR'
regions_df[regions_df == "United States of America"] = 'USA'
regions_df[regions_df == "Viet Nam "] = 'VNM'
df_eng.Region = regions_df

In [19]:
curpol = "GP_CurPol_T45"
ndc = "GP_NDC2030_T45"
nz = "GP_Glasgow"
df_eng_ref = df_eng[['Model', 'Scenario', 'Region']+list(df_eng.keys()[5:])]
df_eng_ref = df_eng_ref[df_eng_ref.Scenario.isin([curpol, ndc, nz])]
scen = np.array(df_eng_ref.Scenario)
scen[scen == ndc] = 'NDC'
scen[scen == curpol] = 'CurPol'
scen[scen == nz] = 'NetZero'
reg = np.array(df_eng_ref.Region)
reg[reg == 'World'] = 'WORLD'
df_eng_ref['Scenario'] = scen
df_eng_ref['Region'] = reg
dummy = df_eng_ref.melt(id_vars=["Scenario", "Model", "Region"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Scenario", "Model", "Region", "Time"])
xr_eng = xr.Dataset.from_dataframe(dummy)
xr_eng = xr_eng.reindex(Time = np.arange(1850, 2101))
xr_eng = xr_eng.interpolate_na(dim="Time", method="linear")

## Plot

In [20]:
for REG in ['IRL']:
    xr_nld = xr.open_dataset("K:/ECEMF/T5.2/Allocations/xr_alloc_"+REG+".nc")
    fig = make_subplots(rows=1, cols=1,
                        #subplot_titles=["<b>Allocations over time", "<b>Full-century budgets"],
                        horizontal_spacing = 0.12, vertical_spacing=0.06)

    # Lineplots
    fig.add_traces(go.Scatter(x=xr_dataread.Time.sel(Time=np.arange(1990, 2022)),
                            y=xr_dataread.sel(Region=REG).GHG_hist.sel(Time=np.arange(1990, 2022)),
                            name='Historic emissions',
                            line={'color': 'black', 'width': 5},
                            mode='lines',
                            showlegend=True), 1, 1)
    fig.add_traces(go.Scatter(x=xr_dataread.Time.sel(Time=np.arange(1990, 2101)),
                            y=[0]*len(xr_dataread.Time.sel(Time=np.arange(1990, 2101))),
                            line={'color': 'black', 'width': 0.5},
                            mode='lines',
                            showlegend=False), 1, 1)
                            
    for rule_i, rule in enumerate(rules_b[::-1]):
        rulecol = rulecolors[::-1][rule_i]
        col = "rgba("+str(webcolors.name_to_rgb(rulecol)[0])+','+str(webcolors.name_to_rgb(rulecol)[1])+','+str(webcolors.name_to_rgb(rulecol)[2])+',0.25)'
        fig.add_traces(go.Scatter(x=xr_nld.Time,
                                y=xr_nld.sel(Temperature=1.5, Risk=[0.5]).min(dim=['NegEmis', 'TrajUnc', 'Risk', 'Scenario', 'Convergence_year'])[rule],
                                name=rulenames[::-1][rule_i],
                                line={'color': col, 'width': 0},
                                mode='lines',
                                showlegend=False))
        fig.add_traces(go.Scatter(x=xr_nld.Time,
                                y=xr_nld.sel(Temperature=1.5, Risk=[0.5]).max(dim=['NegEmis', 'TrajUnc', 'Risk', 'Scenario', 'Convergence_year'])[rule],
                                name=rulenames[::-1][rule_i],
                                line={'color': 'black', 'width': 0},
                                mode='lines',
                                opacity=0.3,
                                fill='tonexty',
                                fillcolor=col,
                                showlegend=False))

    for rule_i, rule in enumerate(rules_b[::-1]):
        rulecol = rulecolors[::-1][rule_i]
        col = "rgba("+str(webcolors.name_to_rgb(rulecol)[0])+','+str(webcolors.name_to_rgb(rulecol)[1])+','+str(webcolors.name_to_rgb(rulecol)[2])+',0.25)'
        fig.add_traces(go.Scatter(x=xr_nld.Time,
                                y=xr_nld.sel(Temperature=1.5, Risk=[0.5]).median(dim=['NegEmis', 'TrajUnc', 'Risk', 'Scenario', 'Convergence_year'])[rule],
                                name=rulenames[::-1][rule_i],
                                line={'color': rulecolors[::-1][rule_i], 'width': 5},
                                mode='lines',
                                showlegend=True))
        
    # # Full-century budgets
    # for rule_i, rule in enumerate(rules_b):
    #     for T_i, T in enumerate([1.5, 2.0]):
    #         vals = np.array(xr_nld.sel(Temperature=T, Risk=[0.5, 0.67][T_i])[rule].sum(dim='Time')).flatten()/1e3
    #         vals[vals == 0] = np.nan
    #         fig.add_traces(go.Violin(x=[rule]*len(vals),
    #                                 y=vals,
    #                                 showlegend=False,
    #                                 points=False,
    #                                 opacity=1.0-T_i*0.7,
    #                                 line={'color': 'black', 'width': 3},
    #                                 line_color=rulecolors[rule_i]),
    #                         1, 2
    #                         )

    fig.update_layout(height=800, width=1000, template='simple_white')
    fig.update_yaxes(tickfont=dict(size=22), row=1, col=1, title='GHG emissions over time', titlefont=(dict(size=25)))
    fig.update_xaxes(tickfont=dict(size=22), row=1, col=1, range=(1990, 2100))
    # fig.update_yaxes(tickfont=dict(size=22), row=1, col=2, title='GHG emission budgets ('+REG+')<br>[Gt CO<sub>2</sub>e]', titlefont=(dict(size=25)))
    # fig.update_xaxes(tickfont=dict(size=22), row=1, col=2)
    fig.update_layout(legend=dict(x=0.01, y=0.01, xanchor='left', yanchor='bottom', font={'size':14}))
    fig.update_annotations(font_size=25)
    fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Pathways_"+REG+".png", scale=3)
    fig.show()